Add import of "connect_many_to_one".

In [1]:
import mosaik
from mosaik.util import connect_randomly, connect_many_to_one

Add a data base to the list of components.

In [2]:
sim_config = {
    'PyPower': {
        'python': 'mosaik_pypower.mosaik:PyPower'
    },
    'CSV': {
        'python': 'simulators.ws2017_csv_sim:CSV'
    },
    'DB': {
        'cmd': 'mosaik-hdf5 %(addr)s'
    }
}

Nothing changed here.

In [3]:
END = 24 * 60 * 60 # one day in seconds
START = '2014-01-01 00:00:00'
PROFILE_DATA = 'data/mprofiles.txt' # .data in your setup
GRID_FILE = 'data/demo_lv_grid.json'

Set up the "world" of the scenario.

In [4]:
world = mosaik.World(sim_config)

Initialize the simulators.

In [5]:
pypower = world.start('PyPower', step_size=15*60)
hhsim = world.start('CSV', sim_start=START, datafile=PROFILE_DATA)

Starting "PyPower" as "PyPower-0" ...
Starting "CSV" as "CSV-0" ...


Instantiate model entities.

In [6]:
grid = pypower.Grid(gridfile=GRID_FILE).children
houses = hhsim.House.create(50)

nodes = [element for element in grid if 'node' in element.eid]

Connect model entities.

In [7]:
connect_randomly(world, houses, nodes, 'P')

{Entity('PyPower-0', '0-node_a1', 'PyPower', PQBus, [], <mosaik.simmanager.LocalProcess object at 0x7f4c2915fd30>),
 Entity('PyPower-0', '0-node_a2', 'PyPower', PQBus, [], <mosaik.simmanager.LocalProcess object at 0x7f4c2915fd30>),
 Entity('PyPower-0', '0-node_a3', 'PyPower', PQBus, [], <mosaik.simmanager.LocalProcess object at 0x7f4c2915fd30>),
 Entity('PyPower-0', '0-node_b1', 'PyPower', PQBus, [], <mosaik.simmanager.LocalProcess object at 0x7f4c2915fd30>),
 Entity('PyPower-0', '0-node_b10', 'PyPower', PQBus, [], <mosaik.simmanager.LocalProcess object at 0x7f4c2915fd30>),
 Entity('PyPower-0', '0-node_b2', 'PyPower', PQBus, [], <mosaik.simmanager.LocalProcess object at 0x7f4c2915fd30>),
 Entity('PyPower-0', '0-node_b3', 'PyPower', PQBus, [], <mosaik.simmanager.LocalProcess object at 0x7f4c2915fd30>),
 Entity('PyPower-0', '0-node_b4', 'PyPower', PQBus, [], <mosaik.simmanager.LocalProcess object at 0x7f4c2915fd30>),
 Entity('PyPower-0', '0-node_b5', 'PyPower', PQBus, [], <mosaik.simmana

Start and instantiate data base.

In [8]:
db = world.start('DB', step_size=60, duration=END)
hdf5 = db.Database(filename='demo.hdf5')

Starting "DB" as "DB-0" ...


Connect model enities to data base entity.

In [9]:
connect_many_to_one(world, houses, hdf5, 'P')

busses = [elem for elem in grid if elem.type in ('RefBus', 'PQBus')]
branches = [elem for elem in grid if elem.type in ('Transformer', 'Branch')]
connect_many_to_one(world, busses, hdf5, 'P', 'Q', 'Vl', 'Vm', 'Va')
connect_many_to_one(world, branches, hdf5, 'P_from', 'Q_from', 'P_to')

Execute co-simulation.

In [10]:
world.run(until=END)

Starting simulation.
Simulation finished successfully.
